In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [2]:
pd.set_option("display.max_column",100)
pd.set_option("expand_frame_repr",False)
train=pd.read_csv(r"D:\文件\学习\数据集\阿里云安全恶意程序检测\security_train\security_train.csv")
test=pd.read_csv("D:\文件\学习\数据集\阿里云安全恶意程序检测\security_test\security_test.csv")

## 一、查看数据

In [3]:
train.head()

,file_id,label,api,tid,index
0,1,5,LdrLoadDll,2488,0
1,1,5,LdrGetProcedureAddress,2488,1
2,1,5,LdrGetProcedureAddress,2488,2
3,1,5,LdrGetProcedureAddress,2488,3
4,1,5,LdrGetProcedureAddress,2488,4


In [4]:
test.head()

,file_id,api,tid,index
0,1,RegOpenKeyExA,2332,0
1,1,CopyFileA,2332,1
2,1,OpenSCManagerA,2332,2
3,1,CreateServiceA,2332,3
4,1,RegOpenKeyExA,2468,0


In [5]:
#查看数据的重合情况
train_fileids=train["file_id"].unique()
test_fileids=test["file_id"].unique()
len(set(train_fileids)-set(test_fileids))

932

In [6]:
len(set(test_fileids)-set(train_fileids))

0

## 二、特征工程

In [7]:
#利用count()函数和nunique()函数生成特征：反映样本调用api，tid,index的频率信息
def simple_sts_features(df):
    simple_fea=pd.DataFrame()
    simple_fea["file_id"]=df["file_id"].unique()
    simple_fea=simple_fea.sort_values("file_id")
    df_grp=df.groupby("file_id")
    simple_fea["file_id_api_count"]=df_grp["api"].count().values
    simple_fea["file_id_api_nunique"]=df_grp["api"].nunique().values
    simple_fea["file_id_tid_count"]=df_grp["tid"].count().values
    simple_fea["file_id_tid_nunique"]=df_grp["tid"].nunique().values
    simple_fea["file_id_index_count"]=df_grp["index"].count().values
    simple_fea["file_id_index_nunique"]=df_grp["index"].nunique().values
    return simple_fea

In [8]:
#利用mean()函数，min()函数，std()函数,max()函数生成特征：tid,index可认为是数值特征，可提取对应的统计特征
def simple_numerical_sts_features(df):
    simple_numerical_fea=pd.DataFrame()
    simple_numerical_fea["file_id"]=df["file_id"].unique()
    simple_numerical_fea=simple_numerical_fea.sort_values("file_id")
    df_grp=df.groupby("file_id")
    simple_numerical_fea["file_id_tid_mean"]=df_grp["tid"].mean().values
    simple_numerical_fea["file_id_tid_min"]=df_grp["tid"].min().values
    simple_numerical_fea["file_id_tid_max"]=df_grp["tid"].max().values
    simple_numerical_fea["file_id_tid_std"]=df_grp["tid"].std().values

    simple_numerical_fea["file_id_index_mean"]=df_grp["index"].mean().values
    simple_numerical_fea["file_id_index_min"]=df_grp["index"].min().values
    simple_numerical_fea["file_id_index_std"]=df_grp["index"].std().values
    simple_numerical_fea["file_id_index_max"]=df_grp["index"].max().values
    
    return simple_numerical_fea

In [9]:
 %%time
 simple_train_fea1=simple_sts_features(train)

Wall time: 1min 17s


In [10]:
%%time
 simple_test_fea1=simple_sts_features(test)

Wall time: 58.3 s


In [11]:
%%time
simple_train_fea2=simple_numerical_sts_features(train)

Wall time: 9.34 s


In [12]:
%%time
simple_test_fea2=simple_numerical_sts_features(test)

Wall time: 8.53 s


In [15]:
simple_train_fea1.head()

,file_id,file_id_api_count,file_id_api_nunique,file_id_tid_count,file_id_tid_nunique,file_id_index_count,file_id_index_nunique
0,1,6786,116,6786,11,6786,5001
1,2,816,30,816,4,816,204
2,3,463,42,463,1,463,463
3,4,2046,51,2046,9,2046,1028
4,5,10002,65,10002,6,10002,5001


In [16]:
simple_train_fea2.head()

,file_id,file_id_tid_mean,file_id_tid_min,file_id_tid_max,file_id_tid_std,file_id_index_mean,file_id_index_min,file_id_index_std,file_id_index_max
0,1,2537.987622,2488,2812,83.881299,2000.806955,0,1510.694221,5000
1,2,2471.000000,2320,2604,101.506783,101.500000,0,58.925137,203
2,3,2208.000000,2208,2208,0.000000,231.000000,0,133.800847,462
3,4,2430.727273,2284,2980,150.460506,511.012219,0,295.407885,1027
4,5,2549.731254,2500,2676,49.556301,2500.000000,0,1443.736493,5000


In [17]:
#获取标签
train_label=train[['file_id','label']].drop_duplicates(subset=['file_id','label'],keep="first")
test_submit=test[['file_id']].drop_duplicates(subset=['file_id'],keep="first")

In [18]:
train_label.head()

,file_id,label
0,1,5
6786,2,2
7602,3,0
8065,4,0
10111,5,0


In [19]:
test_submit.head()

,file_id
0,1
97,2
1458,3
1474,4
1667,5


In [ ]:
#logloss评估模型（手写，因为logloss不是一个常规的评估指标）
